# CNN with Keras Stater

### Please if this kernel is useful, <font color='red'>please upvote !!</font>

This kernel is based on: [CNN with Keras for Humpback Whale ID](https://www.kaggle.com/anezka/cnn-with-keras-for-humpback-whale-id)

### Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import os
import gc
import sys
import math
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow
from tqdm.autonotebook import tqdm
from random import shuffle

from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import keras.backend as K
from keras.models import Sequential
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

### Reading Data

In [ ]:
train_df = pd.read_csv("../input/happy-whale-and-dolphin/train.csv")
#train_df=train_df.head(n=2000)
train_df.head()

### Confg

In [ ]:
IMAGE_SIZE = 128

BATCH_SIZE= 128

Epochs=3
Learning_rate=0.001

num_folds=5
Selected_fold=1 #1,2,3,4,5 

### Functions

In [ ]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    y = onehot_encoded
    return y, label_encoder

def load_images(x,dataset="train_images"):
    img = image.load_img("../input/happy-whale-and-dolphin/"+dataset+"/"+x, target_size=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x /= 255
    return x

In [ ]:
yyy, label_encoder = prepare_labels(train_df['individual_id'])
yyy.shape

In [ ]:
train_df_g=train_df.groupby('individual_id').size()
train_df_g = train_df_g.to_frame()
train_df_g = train_df_g.rename(columns={train_df_g.columns[0]: 'count_cls'})
train_df=pd.merge(train_df,train_df_g,on='individual_id',how='left')
train_df.head(5)

### n Fold

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
sfolder = StratifiedKFold(n_splits=num_folds,random_state=1,shuffle=True)
train_df["Fold"]="train"
X = train_df[['image']]
y = train_df[['individual_id']]

fold_no = 1
for train, valid in sfolder.split(X,y):
    train,
    if fold_no==Selected_fold:
        train_df.loc[valid, "Fold"] = "valid"
    fold_no += 1
    

train_df["Fold"][train_df.count_cls < 3]="train"

y_t=yyy[train_df[train_df.Fold=="train"].index]
y_v=yyy[train_df[train_df.Fold=="valid"].index]

In [ ]:
gc.collect()

In [ ]:
df_train=train_df[train_df.Fold=="train"]
df_valid=train_df[train_df.Fold=="valid"]

df_train.shape,y_t.shape,df_valid.shape,y_v.shape

In [ ]:
print("Number of classes in Training dataset:",len(df_train.groupby('individual_id').size()))
print("Number of classes in validation dataset:",len(df_valid.groupby('individual_id').size()))

In [ ]:
del train_df

### Dataset

In [ ]:
class Dataset(Sequence):
    def __init__(self,df,yyy=None,is_train=True,batch_size=BATCH_SIZE,shuffle=True):
        self.idx = df["image"].values
        self.paths = df["image"].values 
        self.y = yyy   #df["individual_id_int"].values
        self.is_train = is_train
        self.batch_size = batch_size
        self.shuffle = shuffle
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
   
    def __getitem__(self,ids):
        id_path= self.paths[ids]
        batch_paths = self.paths[ids * self.batch_size:(ids + 1) * self.batch_size]
        
        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]
        
        if self.is_train:
            list_x =  [load_images(x,dataset="train_images") for x in batch_paths]
            batch_X = np.stack(list_x)
            return batch_X,batch_y
        else:
            list_x =  [load_images(x,dataset="test_images") for x in batch_paths]
            batch_X = np.stack(list_x)
            return batch_X
    
    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.idx, self.y))
            shuffle(ids_y)
            self.idx, self.y = list(zip(*ids_y))

In [ ]:
train_dataset = Dataset(df_train,y_t,is_train=True,batch_size=BATCH_SIZE)
valid_dataset = Dataset(df_valid,y_v,is_train=True,batch_size=BATCH_SIZE)
for i in range(1):
    images, label = train_dataset[i]
    print("Dimension of the images is:", images.shape)
    print("label=",label.shape)
    print(label)
    plt.imshow(images[0,:,:,:])
    plt.show()

## Class Weights

In [ ]:
le = LabelEncoder()
labels = df_train["individual_id"]
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(labels),
                                                  labels)
class_weights_dict = dict(enumerate(class_weights))

### Model

In [ ]:
model_save = ModelCheckpoint('./last.h5', 
                             save_best_only = True, 
                             save_weights_only = False,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.0001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, 
                              patience = 3, min_delta = 0.0001, 
                              mode = 'min', verbose = 1)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
def create_model():
    efficientnet_layers = EfficientNetB0(weights='imagenet', 
                                         include_top=False, 
                                         input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
                                         pooling='avg')

    model = Sequential()
    model.add(efficientnet_layers)
    model.add(Dense(yyy.shape[1], activation='softmax'))
    model.compile(optimizer = Adam(lr = Learning_rate),
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])

    return model

model = create_model()
model.summary()

### Training

In [ ]:
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=Epochs,
                    verbose=1,
                    #class_weight=class_weights_dict,
                    callbacks = [model_save, early_stop, reduce_lr]
                   )


### Evaluation

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
del train_dataset
del valid_dataset
gc.collect()

## inference

In [ ]:
test = os.listdir("../input/happy-whale-and-dolphin/test_images")
print(len(test))

In [ ]:
col = ['image']
test_df = pd.DataFrame(test, columns=col)
test_df['predictions'] = ''
#test_df=test_df.head(100)

In [ ]:
test_dataset = Dataset(test_df,is_train=False,batch_size=16)

In [ ]:
for i in range(1):
    images = test_dataset[i]
    print("Dimension of the images is:", images.shape)
    plt.imshow(images[0,:,:,:])
    plt.show()

In [ ]:
predictions = model.predict(test_dataset, verbose=1)

In [ ]:
predictions.shape

In [ ]:
for i, pred in enumerate(predictions):
    p=pred.argsort()[-5:][::-1]
    idx=-1
    s=''
    s1=''
    s2=''
    for x in p:
        idx=idx+1
        if pred[x]>0.5:
            s1 = s1 + ' ' +  label_encoder.inverse_transform(p)[idx]
        else:
            s2 = s2 + ' ' + label_encoder.inverse_transform(p)[idx]
    s= s1 + ' new_individual' + s2
    s = s.strip(' ')
    test_df.loc[i, 'predictions'] = s

In [ ]:
test_df.to_csv('submission.csv',index=False)
test_df.head(30)